# Assignment 1: k-nearest neighbors

Only use the already imported libraries `numpy` and `matplotlib.pyplot` for the assignment. Do not import any other library. 

In [ ]:
# Load required packages and dataset. Do not modify.
import matplotlib.pyplot as plt
import numpy as np


def load_iris_dataset():
    from sklearn import datasets
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    return X, y
    
X, y = load_iris_dataset()


## Task 1: Visualization and Preprocessing

1) Explain the content of the dataset in few words. What are the input features? What is the classification target? Check out: [https://en.wikipedia.org/wiki/Iris_flower_data_set](https://en.wikipedia.org/wiki/Iris_flower_data_set).


It is a analysis of Iris setosa, Iris virginica and Iris versicolor. From each of them 50 samples are taken.
There are two features for the length and the width of the sepals and two features for the length and the width of the petals in centimeters.

The classification target is distinguishing the species from each other.

2) Compute and print the following statistics about the dataset:
  - Number of samples
  - Number of samples per class
  - Mean and standard deviation for each input feature


In [ ]:
number_of_samples = len(y)

number_of_samples_0 = np.bincount(y)[0]
number_of_samples_1 = np.bincount(y)[1]
number_of_samples_2 = np.bincount(y)[2]

mean = np.mean(X, axis=0)
standard_deviation = np.std(X, axis=0)

3) Visualize the variables Sepal length and Petal length in a scatter plot (Sepal length on the x-axis, petal length on the y-axis). Color each point of the plot according to its class.

In [ ]:
plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.get_cmap(name="Paired"))
plt.show()

4) Split the dataset randomly into training and test data. 70% of data should be used for training and 30% should be used for testing. Implement the function `train_test_split`. Do not modify the interface of the function.

In [ ]:
def train_test_split(X, y):
    """
    Returns X_train, X_test, y_train, y_test, 
        where X_train and X_test are the input features of the training and test set,
        and y_train and y_test are the class labels of the training and test set.
    """
    train, test = np.split(np.random.permutation(np.c_[X, y]), [int(len(y) * 0.7)])


    return train[:, :-1], test[:, :-1], train[:, -1], test[:, :-1]

X_train, X_test, y_train, y_test = train_test_split(X, y)

assert (X_train.shape[0] + X_test.shape[0]) == X.shape[0]
assert (y_train.shape[0] + y_test.shape[0]) == y.shape[0]
assert X_train.shape[1] == X_test.shape[1]


5) kNN uses a distance measure to identify close neighbors. If the input features are not of the same scale, the distance is not as meaningful, which can negatively impact classification performance. Perform min-max scaling (i.e. scale the values of the input features in such a way that their range is from 0 to 1) on the training and test data. Remember that you should only use information from the training data to perform the scaling on both data sets.

In [ ]:
def min_max(x_train, x_toscale):
    
    X_scaled = (x_toscale - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))    
    return X_scaled  

X_test = min_max(X_train, X_test)
X_train = min_max(X_train, X_train)

## Task 2: k-nearest neighbors

**For B.Sc. Data Science:**  
Implement the kNN algorithm with uniform weighting and arbitrary `k`. Fill out the `predict` method of class `KNearestNeighborsClassifier`. 

Use Euclidean distance to determine the nearest neighbors.
You can ignore the optional parameter `distance_metric`, which is provided as a field in the kNN class.

**For everyone else:**  
Implement the kNN algorithm with distance-based weighting and arbitrary `k`.
Fill out the `predict` method of class `KNearestNeighborsClassifier`.

The parameter `distance_metric` will either contain the string `uniform` or a function. If the value is `uniform`, the classifier should use the Euclidean distance for determining nearest neighbors and uniform weighting. If the value is a function, the classifier should use the function as distance metric and perform distance-weighted classification. An example distance metric is provided with `euclidean_distance`.

In [ ]:
class KNearestNeighbors(object):
    def __init__(self, k, distance_metric='uniform'):
        self.k = k
        self.distance_metric = distance_metric
        
    def fit(self, X, y):
        """
        This functions saves the training data to be used during the prediction.
        """
        self.X = X
        self.y = y
    
    def predict(self, X):
        """
        Returns a vector of shape (n,) if X has shape (n,d), 
        where n is the number of samples and d is the number of features.
        """
        y_pred = []
               
        for sample in X:            
            # entry of form [index, distance, label, weight]
            distance = []
            for i in range(len(self.X)):                
                dist = euclidean_distance(self.X[i], sample) if self.distance_metric == 'uniform' else self.distance_metric(self.X[i],sample)                
                distance.append([i,dist,int(self.y[i]), 0])                
            distance.sort(key=lambda x: x[1])
            distance = distance[:self.k]

            # Add weights depending on distance-rank (harmonic row)
            harmonic_row = [float(1)/float(i+1) for i in range(self.k)]            
            for i in range(self.k):                
                distance[i][3] = sum(harmonic_row[i:]) / self.k                           
                

            if self.distance_metric == 'uniform':                
                occurences = np.bincount(np.array(distance)[:,2].astype(int))
                y_pred.append(np.argmax(occurences))
            else :
                # Determine label by adding weights of each label
                labels = np.unique(np.array(distance)[:,2].astype(int))
                weights = []                
                for lb in labels: 
                    weight = 0                                      
                    for i in range(self.k):
                        if (distance[i][2] == lb): weight = weight + distance[i][3] 
                    weights.append(weight)                
                y_pred.append(labels[np.argmax(weights)])              
        
        return np.array(y_pred)        

    
def euclidean_distance(x1, x2):
    """
    Given vectors x1 and x2 with shape (n,) returns distance between vectors as float.
    """
    return np.sqrt(np.sum((x1 - x2)*(x1 - x2)))



## Task 3: Evaluation

1) Implement functions to compute precision, recall and F1-score. `y_pred` and `y_true` are the vectors of predicted and true class labels respectively with shape `(n,)`, where `n` is the number of samples. Each function should return a float containing the corresponding score.

In [ ]:
def precision(y_pred, y_true):
    # Implement your solution here.
    pass

def recall(y_pred, y_true):
    # Implement your solution here.
    pass

def f1score(y_pred, y_true):
    # Implement your solution here.
    pass

2) Evaluate the performance of kNN with uniform weighting on the Iris dataset for `k=1,3,5`. Train each of the `3` classifiers on the training data from Task 1. Perform the predictions on both the training and test data. Then compute precision, recall, and F1-score for each model and for both training and test data. Print all scores per model. What do you observe?

**For all students other than B.Sc. Data Science:** 
Evaluate the kNN classifier with Euclidean distance-weighting. Compare the performance to uniform-weighting. How does the performance change compared to uniform weighting for each `k`?

In [ ]:
# Implement your solution here.

==> *Write your observations here and report your results.* (double klick here to edit)

3) Explain why kNN with `k=1` achieves perfect results on the training data. Why is it not the best model?

==> *Write your response here.* (double klick here to edit)